# Conduct a Vector Query

> This topic describes how to conduct a vector query.

# Connect to milvus.

In [1]:
#r "nuget: Milvus.Client, 2.2.2-preview.2"
#!import config/Settings.cs

using Milvus.Client;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

//Connect to milvus
(string endpoint, int port, string userName, string password) = Settings.LoadFromFile();

MilvusClient milvusClient = default;

milvusClient = new MilvusClient(endpoint, port, userName, password, null);
milvusClient

Installed Packages Milvus.Client, 2.2.2-preview.2

Address,localhost:19530


# Load collection

🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺🍺

All search and query operations within Milvus are executed in memory. Load the collection to memory before conducting a vector query.

In [2]:
string collectionName = "book";
MilvusCollection collection = milvusClient.GetCollection(collectionName);

if(!(await milvusClient.HasCollectionAsync(collectionName)))
{
    Console.WriteLine($"Collection {collectionName} not exist");
    return;
}

await collection.LoadAsync();

//Waiting for collection loaded
collection.WaitForCollectionLoadAsync(timeout: TimeSpan.FromSeconds(10));

MilvusCollectionDescription collectionInfo = await collection.DescribeAsync();
collectionInfo

Milvus.Client.MilvusCollectionDescription Aliases [ ] CollectionName book CollectionId 443732094175944322 ConsistencyLevel Session CreationTimestamp 1692705169260 Schema Milvus.Client.CollectionSchema Name book Description Fields index value 0 Milvus.Client.FieldSchema Name book_id DataType Int64 IsPrimaryKey True AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension <null> State FieldCreated FieldId 100 1 Milvus.Client.FieldSchema Name word_count DataType Int64 IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension <null> State FieldCreated FieldId 101 2 Milvus.Client.FieldSchema Name book_name DataType VarChar IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength 256 Dimension <null> State FieldCreated FieldId 102 3 Milvus.Client.FieldSchema Name book_intro DataType FloatVector IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension 2 State FieldCreated FieldId 103 EnableDynamicFields False ShardsNum 1 StartPositions (empty)

# Conduct a vector query

The following example filters the vectors with certain book_id values, and returns the book_id field and book_intro of the results.

In [3]:
string expr = "book_id in [2,4,6,8]";

QueryParameters queryParameters = new ();
queryParameters.OutputFields.Add("book_id");
queryParameters.OutputFields.Add("word_count");

IReadOnlyList<FieldData> queryResult = await collection.QueryAsync(
    expr,
    queryParameters);

queryResult

index value 0 Field: {FieldName: book_id, DataType: Int64, Data: 4, RowCount: 4} Data [ 2, 4, 6, 8 ] RowCount 4 FieldName book_id FieldId 0 DataType Int64 IsDynamic False 1 Field: {FieldName: word_count, DataType: Int64, Data: 4, RowCount: 4} Data [ 10002, 10004, 10006, 10008 ] RowCount 4 FieldName word_count FieldId 0 DataType Int64 IsDynamic False

| Parameter | Description |
| --------- | ----------- |
| output_fields (optional) | List of names of the fields to return. |
| vectors | Vectors to query. |
| expr | Boolean expression used to filter attribute. Find more expression details in Boolean Expression Rules. |